In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import os
from PIL import Image
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

# ---------- Getting Insight About the Data ----------

In [ ]:
with Image.open('../input/hot-dog/hotdog/test/hotdog/1010.png') as im:
    display(im)

In [ ]:
path = '../input/hot-dog/hotdog/'
img_names = []

for folder, subfolders, filenames in os.walk(path):
    for img in filenames:
        img_names.append(folder+'/'+img)

In [ ]:
len(img_names)

In [ ]:
img_sizes = []
rejected = []

for item in img_names:
    try:
        with Image.open(item) as img:
            img_sizes.append(img.size)
    except:
        rejected.append(item)

In [ ]:
print(len(img_sizes))
print(len(rejected))

In [ ]:
df = pd.DataFrame(img_sizes)

In [ ]:
df[0].describe()

# ---------- Creating the Transformers ----------

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=.5),
    transforms.RandomRotation(20),
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

In [ ]:
# Inverse normilizing the photos (for later use)
inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.225]
)

In [ ]:
# Visualizing transforms
hotdog = Image.open('../input/hot-dog/hotdog/test/hotdog/1015.png')
display(hotdog)

im = train_transform(hotdog)
plt.imshow(np.transpose(im.numpy(),(1,2,0)))

In [ ]:
# Visualizing inverse normalization
inv_im = inv_normalize(im)
plt.imshow(np.transpose(inv_im.numpy(),(1,2,0)))

# ---------- Loading the data ----------

In [ ]:
root = '../input/hot-dog/hotdog'

train_data = datasets.ImageFolder(os.path.join(root, 'train'), transform = train_transform)
test_data = datasets.ImageFolder(os.path.join(root, 'test'), transform = test_transform)

torch.manual_seed(42)

train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader= DataLoader(test_data, batch_size=10)

class_names = train_data.classes

In [ ]:
class_names

In [ ]:
print(len(train_data))
print(len(test_data))

In [ ]:
for images,labels in train_loader:
    break

In [ ]:
images.shape

In [ ]:
im = make_grid(images, nrow=5)

inv_im = inv_normalize(im)

plt.figure(figsize=(12,4))
plt.imshow(np.transpose(inv_im.numpy(),(1,2,0)))

# ---------- Transfer Learning ----------

In [ ]:
resnet18 = models.resnet18(pretrained=True)

In [ ]:
resnet18

In [ ]:
for param in resnet18.parameters():
    param.requires_grad = False

In [ ]:
torch.manual_seed(42)

resnet18.fc = nn.Sequential(nn.Linear(512,64),
                            nn.ReLU(),
                            nn.Dropout(0.5),
                            nn.Linear(64,2),
                            nn.LogSoftmax(dim=1)
                            )

In [ ]:
resnet18

In [ ]:
for param in resnet18.parameters():
    print(param.numel())

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet18.fc.parameters(),lr=0.001)

In [ ]:
import time
start_time = time.time()

epochs = 10

max_trn_batch = 800
max_tst_batch = 300

train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
    trn_corr = 0
    tst_corr = 0
    
    # Run the training batches
    for b, (X_train, y_train) in enumerate(train_loader):
        if b == max_trn_batch:
            break
        b+=1
        
        # Apply the model
        y_pred = resnet18(X_train)
        loss = criterion(y_pred, y_train)
        
        # Tally the number of correct predictions
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print interim results
        if b%200 == 0:
            print(f'epoch: {i}  batch: {b}  loss: {loss.item():10.8f}  accuracy: {trn_corr.item()*100/(10*b):7.3f}%')
                  
    train_losses.append(loss)
    train_correct.append(trn_corr)
                  
    # Run the testing batches
    with torch.no_grad():
        for b, (X_test, y_test) in enumerate(test_loader):
            if b == max_tst_batch:
              break
    
            # Apply the model
            y_val = resnet18(X_test)

            # Tally the number of correct predictions
            predicted = torch.max(y_val.data, 1)[1]
            tst_corr += (predicted == y_test).sum()

    loss = criterion(y_val, y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)
                  
print(f'\nDuration: {time.time() - start_time:.0f} seconds')

# ---------- Results ----------

In [ ]:
print(test_correct[-1].item())

In [ ]:
image_index = 43

im = inv_normalize(test_data[image_index][0])
plt.imshow(np.transpose(im.numpy(), (1,2,0)))

In [ ]:
# Actual class
class_names[test_data[image_index][1]]

In [ ]:
# Predicted class
resnet18.eval()

with torch.no_grad():
    new_pred = resnet18(test_data[image_index][0].view(1,3,224,224)).argmax()
    
class_names[new_pred.item()]

# ---------- Save Model ----------

In [ ]:
torch.save(resnet18.state_dict(), 'hotdogResNet18')

In [ ]:
# Load Data
# model = Model(*args, **kwargs)
# model.load_state_dict(torch.load('hotdogResNet18'))
# model.eval

# ---------- Plot Loss ----------

In [ ]:
plt.plot(train_losses, label='training loss')
plt.plot(test_losses, label='validation loss')
plt.title('Loss at the end of each epoch')
plt.legend();

# ---------- Test Own Image ----------

In [ ]:
# Load Image
image = Image.open('../input/upload/background7.jpg')

im = test_transform(image)
plt.imshow(np.transpose(im.numpy(),(1,2,0)))

In [ ]:
# Predicted class
resnet18.eval()

with torch.no_grad():
    new_pred = resnet18(im.view(1,3,224,224)).argmax()
    
class_names[new_pred.item()]